In [1]:
import requests
import json
import docx2txt
import pandas as pd
import os

from sklearn.feature_extraction.text import TfidfVectorizer

#url to job search API
url = "https://jsearch.p.rapidapi.com/search"

#query criteria 
querystring = {"query":"data analyst data scientist","page":"1","num_pages":"9","date_posted":"week",
               "remote_jobs_only":"true","employment_types":"FULLTIME"}


#keys
headers = {
    "X-RapidAPI-Key": "XXXXXX",
    "X-RapidAPI-Host": "jsearch.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring).json()

In [2]:
#initates count to keep track of columns in dataframe
count = 1

#Keyword dataframe
Keywordlst = pd.DataFrame(columns = ["Keyword"])

#loop through entir list of company job postions
for i in range (len(response['data'])):
   
    #list to temp hold qualifications and responsibilities
    desc = []
     
    #not all job postins have qualifications and responsibilities
    #will try to pull these first if not will pull entire job description
    #job descriptions have exrta word that could effect ATS score that is 
    #way it is not prefered
    try:
        
        Q=response['data'][i]['job_highlights']['Qualifications']
        R=response['data'][i]['job_highlights']['Responsibilities']
        #combines the to into one list 
        desc = Q + R   
    
    except:
        desc = (response['data'][i]['job_description'])
    
    #creates a temp text.txt file to store job description
    f = open('text.txt','w')
    
    #writes job description to file element by element
    for i in desc:
        #if there is a element that can not be read it skips it and goes to the next one
        try:
            f.write(i)
        except:
            continue
    #closes the writing of the file       
    f.close()
    
    #opens the new text file with the job description
    jobDec=open('text.txt').read()
    
    #starts comparison of resume to job description using text vectorization
    text =[jobDec]
    
    #initates tfidf vectorizer removes stop words
    tv = TfidfVectorizer(analyzer='word',stop_words='english')
    tvfit = tv.fit_transform(text)
    
    #gets word list
    wordlisttv = tv.get_feature_names()
    #puts tfidf into array
    countlisttv =tvfit.toarray().sum(axis=0)
    #turns array into diconary
    wordlist=dict(zip(wordlisttv,countlisttv))

    
    #loops through all wordsin diconary
    for i in wordlist:
         
        #if key is not in dataframes then adds it to it the list
        if i in Keywordlst["Keyword"].values:      
            Keywordlst.loc[Keywordlst["Keyword"]==i,count]= wordlist[i]
        else:
            row = {'Keyword':i, count: wordlist[i]}
            Keywordlst = Keywordlst.append(row, ignore_index = True)
    #new column name       
    count += 1
    #removes 'text' file
    os.remove('text.txt')
   

In [3]:
#Replaces missing data with 0 
Keywordlst=Keywordlst.fillna(0)
#gets average of all columns
Keywordlst["Job_Avg"]=Keywordlst.iloc[: , 1:count].mean(axis=1)
#drops all collumns except the average
Keywordlst=Keywordlst.drop(Keywordlst.iloc[:,1:count], axis=1)
#sorts the average column from highest to lowest
Keywordlst=Keywordlst.sort_values(by='Job_Avg', ascending=False)
#resets the indx used to build ranking
Keywordlst.reset_index(inplace = True)
#adds a ranking to the job description at the first column location
Keywordlst.insert(loc = 0, 
                  column = 'Job Desc Rank',
                  value =Keywordlst.index+1 )

In [4]:
#Compare your resume to job search list

#resuem to check against job descriptions

resume = docx2txt.process("King_Resume_ds_2_final.docx")


#starts comparison of resume to job description using text vectorization
Restext =[resume]
    
#initates tfidf vectorizer removes stop words
Restvfit = tv.fit_transform(Restext)
    
#gets word list
wordlistRes = tv.get_feature_names()
#puts tfidf into array
countlistRes =Restvfit.toarray().sum(axis=0)
#turns array into diconary
wordlistRes=dict(zip(wordlistRes ,countlistRes))

#creates dataframe to hold resume TfIdf score
Resume_df = pd.DataFrame(columns = ["Keyword", "Resume Score"])

#loops through all words from resume put into dictonary
for i in wordlistRes:
    #creates row to append to dataframe
    row = {'Keyword':i, "Resume Score": wordlistRes[i]}
    Resume_df = Resume_df.append(row, ignore_index = True)

#sorts keywords by resume score    
Resume_df=Resume_df.sort_values(by='Resume Score', ascending=False)
#Resets index so that a ranking can be doen
Resume_df.reset_index(inplace = True)
#Creates rank column for resume keyword
Resume_df['Rank'] = Resume_df.index+1

In [5]:
#mergers keywordlist and Resume_df into one dataframe
KeywordlstMerge = pd.merge(Keywordlst,Resume_df, on = 'Keyword', how = "outer")

#removes the index column from the keywordlist and resume_df dataframe
KeywordlstMerge=KeywordlstMerge.drop(KeywordlstMerge.columns[[1,4]],axis = 1)
#add column for the number of job descriptions searched
KeywordlstMerge['Job count']= ""
#count of jobs pulled to ket keyword list
KeywordlstMerge.at[0, 'Job count'] = len(response['data'])

In [6]:
#exports keywordlstMerge freeszes first row and does not print index
KeywordlstMerge.to_excel("keyword.xlsx",freeze_panes=(1,0), index=False)
print("job finished")

job finished
